In [ ]:
%matplotlib widget

In [ ]:
import numpy as np
from scipy import stats
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from scipy.stats import vonmises
import warnings

from pathlib import Path
import flammkuchen as fl

In [ ]:
def combine_groups_data(neural_data_groups, stimulus_data_groups):
    """
    Combine data from multiple groups.
    
    Parameters:
    -----------
    neural_data_groups : list of np.ndarray
        List of neural activity matrices, each of shape (time points × neurons)
    stimulus_data_groups : list of np.ndarray
        List of stimulus matrices, each of shape (8 positions × time points)
        
    Returns:
    --------
    combined_activity : np.ndarray
        Combined neural activity matrix
    combined_stimulus : np.ndarray
        Combined stimulus matrix
    """
    # Get total number of neurons and time points
    total_neurons = sum(group.shape[1] for group in neural_data_groups)
    total_timepoints = sum(group.shape[0] for group in neural_data_groups)
    
    # Initialize combined matrices
    combined_activity = np.zeros((total_timepoints, total_neurons))
    combined_stimulus = np.zeros((8, total_timepoints))
    
    # Fill combined matrices
    neuron_idx = 0
    time_idx = 0
    
    for neural_data, stim_data in zip(neural_data_groups, stimulus_data_groups):
        n_timepoints = neural_data.shape[0]
        n_neurons = neural_data.shape[1]
        
        # Add neural activity
        combined_activity[time_idx:time_idx+n_timepoints, 
                         neuron_idx:neuron_idx+n_neurons] = neural_data
        
        # Add stimulus data
        combined_stimulus[:, time_idx:time_idx+n_timepoints] = stim_data
        
        neuron_idx += n_neurons
        time_idx += n_timepoints
    
    return combined_activity, combined_stimulus

def compute_mean_responses(neural_activity, stimulus_data, angles):
    """
    Compute mean neural responses for each stimulus position.
    """
    n_neurons = neural_activity.shape[1]
    n_angles = len(angles)
    mean_responses = np.zeros((n_angles, n_neurons))
    sem_responses = np.zeros((n_angles, n_neurons))
    
    for i, angle in enumerate(angles):
        # Find timepoints where stimulus was at this angle
        stim_times = stimulus_data[i, :] == 1
        
        if np.sum(stim_times) > 0:
            # Compute mean and SEM for each neuron at this angle
            responses = neural_activity[stim_times, :]
            mean_responses[i, :] = np.mean(responses, axis=0)
            sem_responses[i, :] = stats.sem(responses, axis=0)
    
    return mean_responses, sem_responses

def von_mises_function(x, amplitude, kappa, mu, offset):
    """
    Von Mises function for fitting.
    """
    return amplitude * np.exp(kappa * np.cos(x - mu)) / (2 * np.pi * np.i0(kappa)) + offset

def fit_von_mises(angles, responses, sem=None, p0=None):
    """
    Fit von Mises function to neural responses.
    """
    # Convert angles to radians
    angles_rad = np.deg2rad(angles)
    
    # Set initial parameters if not provided
    if p0 is None:
        amplitude_guess = np.ptp(responses)
        offset_guess = np.min(responses)
        mu_guess = angles_rad[np.argmax(responses)]
        kappa_guess = 1.0
        p0 = (amplitude_guess, kappa_guess, mu_guess, offset_guess)
    
    # Set weights based on SEM if provided
    sigma = None if sem is None else sem + 1e-6
    
    # Define bounds to ensure kappa is positive
    bounds = (
        [0, 0, -np.pi, -np.inf],  # lower bounds: amplitude, kappa, mu, offset
        [np.inf, np.inf, np.pi, np.inf]  # upper bounds
    )
    
    try:
        with warnings.catch_warnings():
            warnings.filterwarnings('ignore')
            params, pcov = curve_fit(von_mises_function, angles_rad, responses, 
                                   p0=p0, sigma=sigma, bounds=bounds, maxfev=10000)
        
        # Calculate R-squared
        y_fit = von_mises_function(angles_rad, *params)
        residuals = responses - y_fit
        ss_res = np.sum(residuals ** 2)
        ss_tot = np.sum((responses - np.mean(responses)) ** 2)
        r_squared = 1 - (ss_res / ss_tot)
        
        return params, pcov, r_squared
    
    except RuntimeError:
        return None, None, -1


def assess_fit_quality(params, pcov, r_squared, min_r_squared=0.3):
    """
    Assess the quality of von Mises fit.
    """
    if params is None or pcov is None:
        return False, "Fit failed to converge"
    
    # Extract parameters
    amplitude, kappa, mu, offset = params
    
    # Check R-squared
    if r_squared < min_r_squared:
        return False, f"Poor fit quality (R² = {r_squared:.2f})"
    
    # Check for extreme kappa values
    if kappa > 60:  # Added check for unreasonably high kappa
        return False, f"Extreme kappa value ({kappa:.1f})"
    
    # Check parameter uncertainties
    param_uncertainties = np.sqrt(np.diag(pcov))
    if np.any(param_uncertainties > 2 * np.abs(params)):
        return False, "High parameter uncertainty"
    
    return True, "Pass"

def analyze_neural_tuning(neural_activity, stimulus_data, angles):
    """
    Analyze tuning curves for all neurons.
    """
    # Compute mean responses for each angle
    mean_responses, sem_responses = compute_mean_responses(neural_activity, stimulus_data, angles)
    
    n_neurons = neural_activity.shape[1]
    results = {
        'kappa_values': [],
        'preferred_angles': [],
        'r_squared_values': [],
        'good_fits': [],
        'rejection_reasons': [],
        'all_params': [],
        'neuron_indices': []  # Keep track of original neuron indices
    }
    
    # Analyze each neuron
    for n in range(n_neurons):
        params, pcov, r_squared = fit_von_mises(angles, mean_responses[:, n], 
                                              sem_responses[:, n])
        
        is_good_fit, reason = assess_fit_quality(params, pcov, r_squared)
        
        results['good_fits'].append(is_good_fit)
        results['rejection_reasons'].append(reason)
        
        if is_good_fit:
            amplitude, kappa, mu, offset = params
            results['kappa_values'].append(kappa)
            results['preferred_angles'].append(np.rad2deg(mu))
            results['r_squared_values'].append(r_squared)
            results['all_params'].append(params)
            results['neuron_indices'].append(n)  # Store original index
    
    return results, mean_responses, sem_responses

def plot_results(results, angles, mean_responses, sem_responses):
    """
    Create visualizations of the analysis results.
    """
    print(f"Total neurons: {mean_responses.shape[1]}")
    print(f"Neurons with good fits: {len(results['kappa_values'])}")
    print(f"Percentage of neurons used: {100 * len(results['kappa_values']) / mean_responses.shape[1]:.1f}%")
    
    # Plot distribution of kappa values
    fig1 = plt.figure(figsize=(10, 4))
    
    # Histogram of kappa values
    plt.subplot(121)
    plt.hist(results['kappa_values'], bins=20)
    plt.xlabel('Kappa')
    plt.ylabel('Count')
    plt.title('Distribution of Kappa Values')
    
    # Create polar plot of preferred directions
    ax = plt.subplot(122, projection='polar')
    preferred_angles_rad = np.deg2rad(results['preferred_angles'])
    kappa_values = results['kappa_values']
    
    # Plot points with kappa values as radius
    ax.scatter(preferred_angles_rad, kappa_values)
    
    # Set the theta limits to ±60 degrees
    ax.set_thetamin(-60)
    ax.set_thetamax(60)
    
    # Set radial limits from 0 to 50
    ax.set_rticks([10, 20, 30, 40, 50])
    ax.set_rmax(50)
    
    # Customize polar plot appearance
    ax.set_title('Preferred Directions and Kappa Values')
    
    # Add grid lines at ±30° intervals
    ax.set_xticks(np.deg2rad([-60, -30, 0, 30, 60]))
    
    plt.tight_layout()
    plt.show()
    
    # Plot example tuning curves
    n_examples = min(5, len(results['all_params']))
    fig2 = plt.figure(figsize=(15, 3))
    
    for i in range(n_examples):
        plt.subplot(1, n_examples, i+1)
        neuron_idx = np.where(results['good_fits'])[0][i]
        
        # Plot raw data with error bars
        plt.errorbar(angles, mean_responses[:, neuron_idx], 
                    yerr=sem_responses[:, neuron_idx],
                    fmt='o', label='Data')
        
        # Plot fitted curve
        angles_fine = np.linspace(-60, 60, 100)
        angles_fine_rad = np.deg2rad(angles_fine)
        params = results['all_params'][i]
        y_fit = von_mises_function(angles_fine_rad, *params)
        plt.plot(angles_fine, y_fit, 'r-', label='Fit')
        
        plt.xlabel('Angle (degrees)')
        plt.ylabel('Response')
        plt.title(f'Neuron {neuron_idx}\nκ={params[1]:.1f}')
        if i == 0:
            plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    return fig1, fig2

In [ ]:
master =  Path(r"Z:\Hagar and Ot\e0075\habenula")
fish_list = list(master.glob("*_f*"))

In [ ]:
angles = np.linspace(-60, 60, 8)  # 8 positions from -60 to 60 degrees

In [ ]:
fish = fish_list[5] / "suite2p"
planes = list(fish.glob("*00*"))
    
neural_data_groups_l = []
neural_data_groups_r = []
stimulus_data_groups = []

for plane in planes:
    
    traces = fl.load(plane / 'filtered_traces.h5')['undetr']
    regs = fl.load(plane / 'sensory_regressors_cells.h5')['regressors']

    hab_coords_l = fl.load(plane / 'habenula_coords.h5')['lhab_coords']
    hab_coords_r = fl.load(plane / 'habenula_coords.h5')['rhab_coords']

    habenula_traces_l = traces[:,hab_coords_l]
    habenula_traces_r = traces[:,hab_coords_r]
    
    neural_data_groups_l =  neural_data_groups_l + [habenula_traces_l]
    neural_data_groups_r =  neural_data_groups_r + [habenula_traces_r]
    
    stimulus_data_groups = stimulus_data_groups + [regs]

In [ ]:
# First combine all groups
combined_activity, combined_stimulus = combine_groups_data(neural_data_groups_r, stimulus_data_groups)

# Then analyze as before
results, mean_responses, sem_responses = analyze_neural_tuning(combined_activity, 
                                                             combined_stimulus, 
                                                             angles)

# Plot results
fig1, fig2 = plot_results(results, angles, mean_responses, sem_responses)

In [ ]:
file_name = 'von mises fit kappa r_hab 2.jpg'
fig1.savefig(fish.parent / file_name, dpi=300)

file_name = 'von mises fit kappa r_hab 2.pdf'
fig1.savefig(fish.parent / file_name, dpi=300)

file_name = 'von mises example fit.jpg'
fig2.savefig(fish.parent / file_name, dpi=300)

file_name = 'von mises example fit.pdf'
fig2.savefig(fish.parent / file_name, dpi=300)

In [ ]:
def plot_multiple_polar_results(all_results, titles=None):
    """
    Create a figure with multiple polar plots arranged in a 4x4 grid.
    """
    fig = plt.figure(figsize=(11, 11))
    gs = fig.add_gridspec(4, 4)
    
    # Create 4x4 grid
    n_plots = min(len(all_results), 16)
    n_rows = 4
    n_cols = 4
    
    if titles is None:
        titles = [f'Dataset {i+1}' for i in range(n_plots)]
    
    # Create each polar plot
    for i in range(n_plots):
        row = i // n_cols
        col = i % n_cols
        
        # Create polar subplot using gridspec
        ax = fig.add_subplot(gs[row, col], projection='polar')
        
        # Get data for this dataset
        results = all_results[i]
        preferred_angles_rad = np.deg2rad(results['preferred_angles'])
        
        # Ensure no kappa values are less than 1 (for log scale)
        kappa_values = np.maximum(results['kappa_values'], 1)
        
        # Set up the log scale before plotting
        ax.set_rscale('log')
        
        # Set limits with a small buffer
        ax.set_rlim(0.9, 55)  # Slightly below 1 and above 50
        
        # Set radial ticks
        r_ticks = [1, 2, 5, 10, 20, 50]
        ax.set_rticks(r_ticks)
        ax.set_yticklabels([str(x) for x in r_ticks])
        
        # Plot points with kappa values as radius
        ax.scatter(preferred_angles_rad, kappa_values, alpha=0.6, zorder=3, s=1, color='k')
        
        # Set the theta limits to ±60 degrees
        ax.set_thetamin(-60)
        ax.set_thetamax(60)
        
        # Customize tick and grid appearance
        ax.tick_params(axis='both', which='both', colors='black', labelsize=8)
        ax.grid(True, alpha=1, zorder=1)
        
        # Add grid lines at ±30° intervals
        ax.set_xticks(np.deg2rad([-60, -30, 0, 30, 60]))
        
        # Remove the outer boundary line
        ax.spines['polar'].set_visible(False)
    
    # Adjust the spacing between subplots
    gs.update(wspace=0.1, hspace=0.2)
    
    plt.show()
    
    return fig

In [ ]:
all_results = []
titles = ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']

# process each dataset
for path in range(0, len(fish_list)):
    fish = fish_list[path] / "suite2p"
    planes = list(fish.glob("*00*"))

    print(len(planes))
    neural_data_groups_r = []
    stimulus_data_groups = []

    for plane in planes:

        traces = fl.load(plane / 'filtered_traces.h5')['undetr']
        regs = fl.load(plane / 'sensory_regressors_cells.h5')['regressors']

        hab_coords_r = fl.load(plane / 'habenula_coords.h5')['rhab_coords']
        habenula_traces_r = traces[:,hab_coords_r]
        neural_data_groups_r =  neural_data_groups_r + [habenula_traces_r]

        stimulus_data_groups = stimulus_data_groups + [regs]
    
    
    # pool data across planes
    combined_activity, combined_stimulus = combine_groups_data(neural_data_groups_r, stimulus_data_groups)
    
    # analyze data
    results, mean_responses, sem_responses = analyze_neural_tuning(combined_activity, 
                                                                 combined_stimulus, 
                                                                 angles)
    
    # store results
    all_results.append(results)

In [ ]:
fig = plot_multiple_polar_results(all_results, titles)

In [ ]:
fig.savefig(master / 'fig x 250106 small dots log scale a 250115.pdf', dpi=300)

In [ ]:
fig.subplots_adjust(wspace=0.1)
plt.show()

In [ ]:
def calculate_high_kappa_fraction(results):
    """
    Calculate fraction of cells with kappa > 10.
    
    Parameters:
    -----------
    results : dict
        Results dictionary from analyze_neural_tuning
        
    Returns:
    --------
    float
        Fraction of cells with kappa > 10
    """
    kappa_values = np.array(results['kappa_values'])
    n_high_kappa = np.sum(kappa_values > 10)
    fraction = n_high_kappa / len(kappa_values) if len(kappa_values) > 0 else 0
    return fraction

def calculate_all_high_kappa_fractions(all_results):
    """
    Calculate fraction of cells with kappa > 10 for each dataset.
    
    Parameters:
    -----------
    all_results : list of dict
        List of results dictionaries from analyze_neural_tuning
        
    Returns:
    --------
    np.ndarray
        Array of fractions, one per dataset
    """
    fractions = []
    for results in all_results:
        fraction = calculate_high_kappa_fraction(results)
        fractions.append(fraction)
    return np.array(fractions)



In [ ]:
fractions = calculate_all_high_kappa_fractions(all_results)

# Print results
for i, frac in enumerate(fractions):
    print(f"Dataset {i+1}: {frac:.2f} ({frac*100:.1f}%)")

In [ ]:
np.mean(fractions)

In [ ]:
np.std(fractions)

In [ ]:
results, mean_responses, sem_responses = analyze_neural_tuning(habenula_traces_r, regs, angles)

plot_results(results, angles, mean_responses, sem_responses)



In [ ]:
file_name = 'von mises fit kappa r_hab 2.jpg'
fig.savefig(fish.parent / file_name, dpi=300)

file_name = 'von mises fit kappa r_hab 2.pdf'
fig.savefig(fish.parent / file_name, dpi=300)